In [ ]:
# Assume that each person has a limit order distribution given by a N(a, b) for selling and buying

# N = number of actors
# Bid = B_mean, B_std
# Ask = A_mean, A_std

# The orders are inserted in a queue and matched

In [ ]:
import numpy as np
import pandas as pd
import scipy as scipy

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import lib

In [ ]:
print(ask_rv)
print(bid_rv)

In [ ]:
ob = lib.get_data()

display(ob)

ob.plot.hist()

In [ ]:
sd = lib.get_supply_demand_curve(ob)

display(sd)

sd.plot()

In [ ]:
sd.index[4]

In [ ]:
eq_idx = lib.find_equilibrium(sd)
#eq_quantity = (sd.iloc[eq_idx].index.values[0] + sd.iloc[eq_idx + 1].index.values[0]) / 2
print(sd.iloc[eq_idx].index)
eq_quantity1 = sd.index.values[eq_idx]
eq_quantity2 = sd.index.values[eq_idx + 1]
#print(eq_quantity1, eq_quantity2)
eq_quantity = np.mean([eq_quantity1, eq_quantity2])
print(eq_idx, eq_quantity)
ax = sd.plot()
ymin, ymax = ax.get_ylim()
ax.vlines(eq_quantity, ymin=ymin, ymax=ymax, color="r")

In [ ]:
excess = df["supply"] - df["demand"]
excess.plot()
zero_crossings = np.where(np.diff(np.sign(excess)))
idx = zero_crossings[0]
print(idx)
excess.iloc[idx]

In [ ]:
excess
idx = np.where(np.diff(np.sign(excess)) == 2)[0]
excess.iloc[idx + 1]

In [ ]:
a